In [1]:
import pandas as pd
import numpy as np
from random import random, seed

In [2]:
df = pd.read_excel("dataset aluminium lme.xlsx")
df.head()

,Tanggal,LME Aluminium Cash,LME Aluminium 3 Months,LME Alumina 3 Months
0,2020-06-01,1551.5,1540.0,255.75
1,2020-06-02,1499.5,1529.0,255.75
2,2020-06-03,1526.0,1555.5,255.75
3,2020-06-04,1535.0,1560.5,255.75
4,2020-06-05,1553.5,1579.0,255.75


In [3]:
# dm = df.loc[0:6]
dm = df
dm

,Tanggal,LME Aluminium Cash,LME Aluminium 3 Months,LME Alumina 3 Months
0,2020-06-01,1551.5,1540.0,255.75
1,2020-06-02,1499.5,1529.0,255.75
2,2020-06-03,1526.0,1555.5,255.75
3,2020-06-04,1535.0,1560.5,255.75
4,2020-06-05,1553.5,1579.0,255.75
...,...,...,...,...
57,2020-07-28,1671.0,1709.0,264.00
58,2020-07-29,1687.0,1726.5,271.75
59,2020-07-30,1681.5,1719.0,275.00
60,2020-07-31,1685.5,1722.0,275.00


In [4]:
# for i in range(len(dm["LME Aluminium Cash"])):
#     if i == 0:
#         print ((dm["LME Aluminium Cash"][i:(i+1)*3]) )
#         print (max(dm["LME Aluminium Cash"][i:(i+1)*3]))
#         print (min(dm["LME Aluminium Cash"][i:(i+1)*3]))

#         print("target: ", dm["LME Aluminium Cash"][i+3])
#     elif (i)+3 < len(dm["LME Aluminium Cash"]):
#         print ((dm["LME Aluminium Cash"][i:(i)+3] ))
#         print("target: ", dm["LME Aluminium Cash"][i+3])
col = []
for i in dm.columns:
    if i != "Tanggal":
        col.append(i)
def preprocess(column):
    threeDaysData = pd.DataFrame(columns= ["x1", "x2", "y"])
    for i in range(len(dm["LME Aluminium Cash"])):
        if i == 0:
            threeDaysData.loc[i, "x1"] = max(dm[column][i:(i+1)*3])
            threeDaysData.loc[i, "x2"] = min(dm[column][i:(i+1)*3])
            threeDaysData.loc[i, "y"] = dm[column][i+3]
        elif (i)+3 < len(dm[column]):
            threeDaysData.loc[i, "x1"] = max(dm[column][i:(i)+3])
            threeDaysData.loc[i, "x2"] = min(dm[column][i:(i)+3] )
            threeDaysData.loc[i, "y"] = dm[column][i+3]
    return threeDaysData
threeDaysData = preprocess(col[0])
threeDaysData.head()

,x1,x2,y
0,1551.5,1499.5,1535
1,1535,1499.5,1553.5
2,1553.5,1526,1553.5
3,1553.5,1535,1553.5
4,1553.5,1553.5,1569.5


In [5]:
def minmax(data):
    minimum = {}
    maximum = {}
    for i in data.columns:
        minimum[i] = data[i].min()
        maximum[i] = data[i].max()
    for i in data.index:
        for j in data.columns:
            data.loc[i, j] = (data.loc[i,j] - minimum[j]) / (maximum[j] - minimum[j])
    return data
normalData = minmax(threeDaysData)
normalData.head()

,x1,x2,y
0,0.108553,0,0
1,0,0,0.121711
2,0.121711,0.145604,0.121711
3,0.121711,0.195055,0.121711
4,0.121711,0.296703,0.226974


In [6]:
def sigmoid(value):
    return 1 / (1 + np.exp(-value))

In [7]:
def differ_sigmoid(value):
    return sigmoid(value) * (1 - sigmoid(value))

In [8]:
dataIn = normalData.iloc[0]
dataIn

x1    0.108553
x2           0
y            0
Name: 0, dtype: object

In [9]:
b_in = 0.1
layer = 6
b_out = 0.1
alpha = 0.1

In [10]:
weightArray = []
for i in range(2):
    weightArray.append(pd.DataFrame(index=[0]))
for i in range(len(weightArray)):
    for j in range(layer):
#         weightArray[i].loc[0, j] = (j+1) * 0.1
        weightArray[i].loc[0, j] = random()

weightArray[1]

,0,1,2,3,4,5
0,0.527808,0.801632,0.620474,0.207737,0.555741,0.925298


In [11]:
inputBias = []
for i in range(layer):
    inputBias.append(random())

In [12]:
outputLayer = pd.DataFrame(columns=[i for i in range(layer)])
for i in range(len(outputLayer.columns)):
    outputLayer.loc[0, i] = (i+1) * 0.1
outputLayer.loc[0, "b"] = random()
# outputLayer.loc[0, "b"] = 0.1

outputLayer

,0,1,2,3,4,5,b
0,0.1,0.2,0.3,0.4,0.5,0.6,0.185743


In [13]:
# z_net = pd.DataFrame(0, index = [i for i in range(len(dataIn.index))], columns=[i for i in range(layer)])
z_net = []
for i in range(layer):
    z_net.append((inputBias[i] + dataIn["x1"] * weightArray[0].loc[0, i] + dataIn["x2"] * weightArray[1].loc[0, i]
))
z_net

[0.9518154311403181,
 0.0734307495684178,
 0.4424522316187576,
 0.6587287180861378,
 0.45693153739219833,
 1.0441929253805933]

In [14]:
z_activate = []
#         z_activate.loc[i, j] = 1 / (1+(2.71828183 ** -(z_net.loc[i, j])))
for i in range((layer)):
    z_activate.append(sigmoid(z_net[i]))

z_activate

[0.7214801292915753,
 0.5183494430019364,
 0.6088431930791915,
 0.6589747548115035,
 0.6122859995722423,
 0.7396582232673263]

In [15]:
y_net = []
y_temp = []

for i in range(layer):
    y_temp.append((z_activate[i] * outputLayer.loc[0, i]))
y_net.append(np.sum(y_temp)+b_out)
y_net

[1.4719986951244208]

In [16]:
y_activate = []
for i in y_net:
    y_activate.append(sigmoid(i))
y_activate

[0.8133609877768121]

In [17]:
error = dataIn["y"] - y_activate[0]
print(error)
mse = (error ** 2)
mse

-0.8133609877768121


0.6615560964372714

In [18]:
backOutputTarget = []
for i in range(len(y_activate)):
    backOutputTarget.append((dataIn["y"] - 
                             y_activate[i]) * differ_sigmoid(y_net[i]))
backOutputTarget            

[-0.12347217636928039]

In [19]:
backAlphaHidden = []
for i in range(layer):
    backAlphaHidden.append(alpha * backOutputTarget[0] * z_activate[i])
backAlphaHidden.append(alpha * backOutputTarget[0] * b_out)
backAlphaHidden

[-0.008908272177082061,
 -0.006400173384725334,
 -0.007517519411710977,
 -0.008136504714898926,
 -0.007560028492762503,
 -0.009132721059625188,
 -0.001234721763692804]

In [20]:
errorTarget = []
for i in range(layer):
    errorTarget.append(backOutputTarget[0] * outputLayer.loc[0, i])
errorTarget

[-0.012347217636928039,
 -0.024694435273856077,
 -0.03704165291078412,
 -0.049388870547712155,
 -0.061736088184640193,
 -0.07408330582156825]

In [21]:
errorHidden = []
for i in range(layer):
    errorHidden.append(errorTarget[i] * differ_sigmoid(z_net[i]))
errorHidden

[-0.002481130814996352,
 -0.006165294151274278,
 -0.008821586667150182,
 -0.011099014061503055,
 -0.014655644451383908,
 -0.014265772962400369]

In [22]:
deltaInputHidden = []
deltaInputBias = []
for i in range(2):
    deltaInputHidden.append(pd.DataFrame(index=[i for i in range(1)]))
for i in range(len(deltaInputHidden)):
    for j in range(layer):
        deltaInputHidden[i].loc[0, j] = alpha * errorHidden[j] * dataIn[i]
for i in range(layer):
    deltaInputBias.append(alpha * errorHidden[i] * inputBias[i])
deltaInputHidden[0]

,0,1,2,3,4,5
0,-0.000027,-0.000067,-0.000096,-0.00012,-0.000159,-0.000155


In [23]:
newWeightArray = []
inputBiasNew = []
for i in range(2):
    newWeightArray.append(pd.DataFrame(index=[i for i in range(1)]))
for i in range(len(newWeightArray)):
    for j in range(layer):
        newWeightArray[i].loc[0, j] = deltaInputHidden[i].loc[0, j] + weightArray[i].loc[0, j]
for i in range(layer):
    inputBiasNew.append(inputBias[i] - deltaInputBias[i])

newWeightArray[1]

,0,1,2,3,4,5
0,0.527808,0.801632,0.620474,0.207737,0.555741,0.925298


In [24]:
newOutputLayer = pd.DataFrame(index = [i for i in range(1)], columns=[i for i in range(layer)])
for i in newOutputLayer.index:
    for j in newOutputLayer.columns:
        newOutputLayer.loc[i, j] = backAlphaHidden[j] + outputLayer.loc[0, j]
for i in newOutputLayer.index:
    newOutputLayer.loc[i, "b"] = backAlphaHidden[len(backAlphaHidden)-1] + b_out
newOutputLayer

,0,1,2,3,4,5,b
0,0.0910917,0.1936,0.292482,0.391863,0.49244,0.590867,0.098765


In [25]:
def back_train(x, epoch, alpha, layer, max_mse):
    et = 0
    ep = 0
    weightArray = []
    outputLayer = pd.DataFrame(columns=[i for i in range(layer)])
    inputBias = []

    mse = 1
    while (epoch == -1 or ep <  epoch) and mse > max_mse:

        print("MSE: ", mse, "Epoch: ", ep)
        dataIn = x.iloc[et]
        if ep == 0:
            for i in range(2):
                weightArray.append(pd.DataFrame(index=[0]))
            for i in range(len(weightArray)):
                for j in range(layer):
                    weightArray[i].loc[0, j] = random()
            for i in range(layer):
                inputBias.append(random())
            for i in range(len(outputLayer.columns)):
                outputLayer.loc[0, i] = random()
            outputLayer.loc[0, "b"] = random()
        else:
            inputBias = inputBiasNew
            weightArray = newWeightArray
            outputLayer = newOutputLayer
        z_net = []
        for i in range(layer):
            z_net.append((inputBias[i] + dataIn["x1"] * weightArray[0].loc[0, i] + dataIn["x2"] * weightArray[1].loc[0, i]
        ))
        z_activate = []
        #         z_activate.loc[i, j] = 1 / (1+(2.71828183 ** -(z_net.loc[i, j])))
        for i in range((layer)):
            z_activate.append(sigmoid(z_net[i]))
        y_net = []
        y_temp = []

        for i in range((layer)):
            y_temp.append((z_activate[i] * outputLayer.loc[0, i]))
        y_net.append(np.sum(y_temp)+b_out)
        y_activate = []
        for i in y_net:
            y_activate.append(sigmoid(i))
        error = dataIn["y"] - y_activate[0]
        mse = (error ** 2)
        backOutputTarget = []
        for i in range(len(y_activate)):
            backOutputTarget.append((dataIn["y"] - 
                                     y_activate[i]) * differ_sigmoid(y_net[i]))
        backAlphaHidden = []
        for i in range(layer):
            backAlphaHidden.append(alpha * backOutputTarget[0] * z_activate[i])
        backAlphaHidden.append(alpha * backOutputTarget[0] * b_out)
        errorTarget = []
        for i in range(layer):
            errorTarget.append(backOutputTarget[0] * outputLayer.loc[0, i])
        errorHidden = []
        for i in range(layer):
            errorHidden.append(errorTarget[i] * differ_sigmoid(z_net[i]))
        deltaInputHidden = []
        deltaInputBias = []
        for i in range(2):
            deltaInputHidden.append(pd.DataFrame(index=[i for i in range(1)]))
        for i in range(len(deltaInputHidden)):
            for j in range(layer):
                deltaInputHidden[i].loc[0, j] = alpha * errorHidden[j] * dataIn[i]
        for i in range(layer):
            deltaInputBias.append(alpha * errorHidden[i] * inputBias[i])
        newWeightArray = []
        inputBiasNew = []
        for i in range(2):
            newWeightArray.append(pd.DataFrame(index=[i for i in range(1)]))
        for i in range(len(newWeightArray)):
            for j in range(layer):
                newWeightArray[i].loc[0, j] = deltaInputHidden[i].loc[0, j] + weightArray[i].loc[0, j]
        for i in range(layer):
            inputBiasNew.append(inputBias[i] - deltaInputBias[i])
        newOutputLayer = pd.DataFrame(index = [i for i in range(1)], columns=[i for i in range(layer)])
        for i in newOutputLayer.index:
            for j in newOutputLayer.columns:
                newOutputLayer.loc[i, j] = backAlphaHidden[j] + outputLayer.loc[0, j]
        for i in newOutputLayer.index:
            newOutputLayer.loc[i, "b"] = backAlphaHidden[len(backAlphaHidden)-1] + b_out
        
        ep += 1

        if et < len(dataIn):
            et += 1
        elif et > len(dataIn) - 1:
            et = 0
    return weightArray, outputLayer, inputBias

In [26]:
def back_test(x, weightArray, outputLayer, inputBias, layer):
    result = []
    for m in range(len(x)):
        dataIn = x.iloc[m]
        z_net = []
        for i in range(layer):
            z_net.append((inputBias[i] + dataIn["x1"] * weightArray[0].loc[0, i] + dataIn["x2"] * weightArray[1].loc[0, i]
        ))
        z_activate = []
        #         z_activate.loc[i, j] = 1 / (1+(2.71828183 ** -(z_net.loc[i, j])))
        for i in range(layer):
            z_activate.append(sigmoid(z_net[i]))
        y_net = []
        y_temp = []

        for i in range(layer):
            y_temp.append((z_activate[i] * outputLayer.loc[0, i]))
        y_net.append(np.sum(y_temp)+b_out)
        y_activate = []
        for i in y_net:
            y_activate.append(sigmoid(i))
            result.append(sigmoid(i))
    return result



In [92]:
seed(1)
train = normalData.iloc[0:50]
test = normalData.iloc[50:]
layer = 12
weightArray, outputLayer, inputBias = (back_train(train, 10, 0.1, layer, .1))

MSE:  1 Epoch:  0
MSE:  0.9757409836880977 Epoch:  1
MSE:  0.7480546645312949 Epoch:  2
MSE:  0.7526204839289625 Epoch:  3
MSE:  0.7533143837544595 Epoch:  4
MSE:  0.9753426666538746 Epoch:  5
MSE:  0.7476797528489927 Epoch:  6
MSE:  0.7522981841809789 Epoch:  7
MSE:  0.7529993569653334 Epoch:  8
MSE:  0.9749311236261318 Epoch:  9


In [93]:
hasil = back_test(test, weightArray, outputLayer, inputBias, layer)
hasil

[0.9958212940192389,
 0.9958831527342391,
 0.9959302190940342,
 0.9958283990896717,
 0.9959378876199197,
 0.9960792987714427,
 0.9963796779174215,
 0.9964830887102838,
 0.9965914475521509]

In [94]:
from sklearn.metrics import mean_absolute_error 
from sklearn.metrics import mean_squared_error 
from math import sqrt

#MAE
mae = mean_absolute_error(test["y"], hasil)
print('MAE: %f' % mae)

#MSE
mse = mean_squared_error(test["y"], hasil)
print('MSE: %f' % mse)

#RMSE
rmse = sqrt(mse)
print('RMSE: %f' % rmse)

#MAPE (Mean Absolute Percentage Error)
actual = np.array([test["y"]])
forecast = np.array([hasil])
mape=np.mean(np.abs((actual - forecast) / actual)) * 100
print('MAPE: %f' % mape)

# ACCURACY
Accuracy= 100-mape
print('Accuracy: %f' % Accuracy)

MAE: 0.104797
MSE: 0.018897
RMSE: 0.137468
MAPE: 12.894367
Accuracy: 87.105633


In [95]:
actual, forecast

(array([[0.7861842105263158, 0.7861842105263158, 0.7861842105263158,
         0.8322368421052632, 0.8947368421052632, 1.0, 0.9638157894736842,
         0.9901315789473685, 0.9901315789473685]], dtype=object),
 array([[0.99582129, 0.99588315, 0.99593022, 0.9958284 , 0.99593789,
         0.9960793 , 0.99637968, 0.99648309, 0.99659145]]))